In [7]:
# Import necessary libraries
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from newsapi import NewsApiClient


In [8]:
# Download historical data for Apple (AAPL) for the last 4 years
data = yf.download("AAPL", start="2020-01-01", end="2023-12-31")

[*********************100%***********************]  1 of 1 completed


In [9]:
# Display the first few rows of the data
data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,74.059998,75.150002,73.797501,75.087502,72.876106,135480400
2020-01-03,74.287498,75.144997,74.125000,74.357498,72.167610,146322800
2020-01-06,73.447502,74.989998,73.187500,74.949997,72.742661,118387200
2020-01-07,74.959999,75.224998,74.370003,74.597504,72.400528,108872000
2020-01-08,74.290001,76.110001,74.290001,75.797501,73.565193,132079200


In [10]:
# Function to fetch fundamental data
def fetch_fundamental_data(ticker):
    stock = yf.Ticker(ticker)
    fundamentals = {}
    try:
        fundamentals['PE_Ratio'] = stock.info['trailingPE']
        fundamentals['EPS'] = stock.info['trailingEps']
        fundamentals['Revenue'] = stock.financials.loc['Total Revenue'].iloc[0]
    except Exception as e:
        print("Error fetching fundamental data:", e)
        fundamentals = None
    return fundamentals

# Fetch fundamental data for Apple
fundamental_data = fetch_fundamental_data("AAPL")


In [11]:
# Function to fetch fundamental data
def fetch_fundamental_data(ticker):
    stock = yf.Ticker(ticker)
    fundamentals = {}
    try:
        fundamentals['PE_Ratio'] = stock.info['trailingPE']
        fundamentals['EPS'] = stock.info['trailingEps']
        fundamentals['Revenue'] = stock.financials.loc['Total Revenue'].iloc[0]
    except Exception as e:
        print("Error fetching fundamental data:", e)
        fundamentals = None
    return fundamentals

# Fetch fundamental data for Apple
fundamental_data = fetch_fundamental_data("AAPL")

In [13]:
# Function to perform sentiment analysis on news headlines
def get_sentiment_from_news(news_list):
    analyzer = SentimentIntensityAnalyzer()
    sentiment_score = 0
    for news in news_list:
        sentiment = analyzer.polarity_scores(news)['compound']
        sentiment_score += sentiment
    return sentiment_score / len(news_list) if news_list else 0

In [14]:
# Initialize the NewsAPI client with your API key
newsapi = NewsApiClient(api_key='45144efccea34226afed83713f2c6547')  # Replace 'YOUR_API_KEY' with your actual NewsAPI key

# Function to get news headlines from NewsAPI
def fetch_news_headlines(ticker):
    # Use NewsAPI to fetch news articles related to the ticker symbol
    all_articles = newsapi.get_everything(q=ticker,
                                          language='en',
                                          sort_by='relevancy',
                                          page_size=5)  # Adjust the number of articles as needed
    
    # Extract headlines from the articles
    headlines = [article['title'] for article in all_articles['articles']]
    
    return headlines

# Example usage
news_headlines = fetch_news_headlines("AAPL")
print(news_headlines)

In [15]:
# Fetch and analyze sentiment of news headlines
news_headlines = fetch_news_headlines("AAPL")
news_sentiment = get_sentiment_from_news(news_headlines)